In [1]:
%matplotlib notebook
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

# 1.1Load Data

In [2]:
homedir = os.environ['HOME']

train = pd.read_csv(homedir+'/data/mangaki-data-challenge/train.csv')
unsuper = pd.read_csv(homedir+'/data/mangaki-data-challenge/watched.csv')
title = pd.read_csv(homedir+'/data/mangaki-data-challenge-0908/titles.csv')
scraping = pd.read_csv(homedir+'/data/mangaki-data-challenge/scraping_data.csv', index_col=0)
test = pd.read_csv(homedir+'/data/mangaki-data-challenge/test.csv')

In [3]:
scraping.head()

img category                            title
0    1    anime                       Death Note
1    0    anime  Code Geass: Hangyaku no Lelouch
2    1    anime                 Sword Art Online
3    1    anime                           Naruto
4    1    anime             L'Attaque des Titans

# 1.2Merge and Sort Data

In [4]:
unsuper = pd.merge(unsuper, title, on="work_id")
unsuper = pd.merge(unsuper, scraping, on=['title', 'category'], how='left')

train = train.sort_values(by=["user_id", "work_id"])
unsuper = unsuper.sort_values(by=["user_id", "work_id"])

# 2.FE
## 2.1 fill Na, label encoding

In [5]:
# fill Na
unsuper['img'] = unsuper['img'].fillna(-1).astype(int)

# label encoding
label_dict = {'dislike':0, 'neutral':1, 'like':2, 'love':3}
unsuper['rating_id'] = unsuper['rating'].apply(lambda x: label_dict[x])

label_dict = {'album':0, 'anime':1, 'manga':2}
unsuper['category_id'] = unsuper['category'].apply(lambda x: label_dict[x])

unsuper.drop(['rating', 'category', 'title'], axis = 1, inplace=True)

unsuper.head()

user_id  work_id  img  rating_id  category_id
52695        0       79    1          2            1
7431         0      104    1          2            1
67192        0     1701    1          2            1
27042        0     2739    1          2            1
75752        0     3562    1          2            1

## 2.2 FE with two feature quantities
2.2.1 user_id-watched_work_id distribution

2.2.2 user_id-rating distribution

2.2.3 user_id-catecory distribution

2.2.4 work_id-rating distribution

2.2.5 Correspondence of work_id-img(scraping data), work_id-category_id

### 2.2.1 user_id-watched_work_id distribution

In [6]:
pivot_wu_id = unsuper.pivot(index='user_id', columns='work_id', values='work_id').fillna(-1) > 0
pivot_wu_id.head()

work_id   0      1      2      3      4      5      6      7      9      10    \
user_id                                                                         
0        False  False  False  False  False  False  False  False  False  False   
1        False  False  False  False  False  False  False  False  False  False   
2        False  False   True  False  False  False  False  False  False  False   
3        False  False   True  False  False  False  False  False  False  False   
4        False  False   True  False  False  False  False  False  False  False   

work_id  ...     9885   9886   9887   9889   9890   9891   9892   9893   9894  \
user_id  ...                                                                    
0        ...    False  False  False  False  False  False  False  False  False   
1        ...    False  False  False  False  False  False  False  False  False   
2        ...    False  False  False  False  False  False  False  False  False   
3        ...    False  False  False  False  False  False  False  False  False   
4        ...    False  False  False  False  False  False  False  False  False   

work_id   9896  
user_id         
0        False  
1        False  
2        False  
3        False  
4        False  

[5 rows x 8584 columns]

In [7]:
from sklearn.manifold import TSNE
TSNE_pivot_wu_id = TSNE(n_components=3, perplexity=5).fit_transform(pivot_wu_id)

from sklearn.decomposition import PCA
PCA_pivot_wu_id = PCA(n_components=5).fit_transform(pivot_wu_id)

from sklearn import (decomposition, ensemble)

hasher = ensemble.RandomTreesEmbedding(n_estimators=200, random_state=0,
                                       max_depth=10)
X_transformed = hasher.fit_transform(pivot_wu_id)
pca = decomposition.TruncatedSVD(n_components=5)
RFE_pivot_wu_id = pca.fit_transform(X_transformed)

UW_ID = pd.DataFrame(np.concatenate([TSNE_pivot_wu_id, PCA_pivot_wu_id, RFE_pivot_wu_id], axis=1))
UW_ID.index = pivot_wu_id.index

### 2.2.2 user_id-rating distribution
### 2.2.3 user_id-catecory distribution

In [8]:
dum1 = pd.get_dummies(unsuper['rating_id'],prefix='rate_u')
dum2 = pd.get_dummies(unsuper['category_id'],prefix='cate_u')
URC_data = dum1.join(dum2).join(unsuper['user_id'])

In [9]:
pt_URC = pd.pivot_table(data=URC_data, index = 'user_id', aggfunc='sum')
pt_URC['count'] = unsuper['user_id'].value_counts().sort_index()

pt_URC.head()

cate_u_0  cate_u_1  cate_u_2  rate_u_0  rate_u_1  rate_u_2  rate_u_3  \
user_id                                                                         
0             0.0      19.0       0.0       1.0       1.0      17.0       0.0   
1             0.0      87.0       0.0      10.0       6.0      33.0      38.0   
2             0.0      49.0       0.0       2.0       2.0      45.0       0.0   
3             0.0     402.0       0.0      23.0      84.0     295.0       0.0   
4             0.0      64.0       9.0       6.0       6.0      35.0      26.0   

         count  
user_id         
0           19  
1           87  
2           49  
3          402  
4           73

In [10]:
pt_URC.describe()

cate_u_0     cate_u_1     cate_u_2     rate_u_0     rate_u_1  \
count  1963.000000  1963.000000  1963.000000  1963.000000  1963.000000   
mean      0.001019    82.695874    18.663271    11.928681    22.513500   
std       0.031911   124.480018    53.439432    32.350259    55.326436   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000    12.000000     0.000000     0.000000     1.000000   
50%       0.000000    35.000000     1.000000     2.000000     6.000000   
75%       0.000000    97.000000    15.500000    10.000000    21.000000   
max       1.000000  1132.000000   948.000000   560.000000  1119.000000   

          rate_u_2    rate_u_3        count  
count  1963.000000  1963.00000  1963.000000  
mean     61.148242     5.76974   101.360163  
std      84.713837    14.07121   144.862936  
min       0.000000     0.00000     1.000000  
25%      10.500000     0.00000    18.000000  
50%      29.000000     0.00000    47.000000  
75%      77.000000     5.00000   122.000000  
max     818.000000   202.00000  1513.000000

In [11]:
# make user-category, user-rating multinomial distribution
# prior is Dirichlet distribution

# pt_URC.describe()
# mean	0.001019	82.695874	18.663271	11.928681	22.513500	61.148242	5.76974	101.360163
alpha_category = np.array([0.0,82.7,18.7])*0.05
alpha_rate = np.array([11.9,22.5,61.1,5.8])*0.05

alpha = np.concatenate([alpha_category, alpha_rate])

In [12]:
URC_dist = pt_URC.drop('count', axis=1).apply(lambda x:x+alpha, axis=1)

count = pt_URC['count']

for i in ['cate_u_0','cate_u_1','cate_u_2']:
    URC_dist[i] = URC_dist[i] / (count+np.sum(alpha_category))

for i in ['rate_u_0','rate_u_1','rate_u_2', 'rate_u_3']:
    URC_dist[i] = URC_dist[i] / (count+np.sum(alpha_rate))

URC_dist.head()

cate_u_0  cate_u_1  cate_u_2  rate_u_0  rate_u_1  rate_u_2  rate_u_3
user_id                                                                      
0             0.0  0.961155  0.038845  0.066279  0.088303  0.833368  0.012051
1             0.0  0.989845  0.010155  0.115082  0.077391  0.391625  0.415902
2             0.0  0.982708  0.017292  0.047998  0.057801  0.888838  0.005364
3             0.0  0.997703  0.002297  0.057964  0.209119  0.732205  0.000712
4             0.0  0.872742  0.127258  0.084481  0.091270  0.487478  0.336771

### 2.2.4 work_id-rating distribution

In [13]:
dum1 = pd.get_dummies(unsuper['rating_id'],prefix='rate_w')
WR_data = dum1.join(unsuper['work_id'])

pt_WR = pd.pivot_table(data=WR_data, index='work_id', aggfunc='sum')
pt_WR['count'] = unsuper['work_id'].value_counts().sort_index()

pt_WR.head()

rate_w_0  rate_w_1  rate_w_2  rate_w_3  count
work_id                                               
0             2.0       2.0       2.0       0.0      6
1             0.0       0.0       1.0       0.0      1
2            50.0      85.0     373.0      41.0    549
3             0.0       1.0       4.0       0.0      5
4             2.0       3.0       8.0       1.0     14

In [14]:
pt_WR.describe()

rate_w_0     rate_w_1     rate_w_2     rate_w_3        count
count  8584.000000  8584.000000  8584.000000  8584.000000  8584.000000
mean      2.727866     5.148416    13.983458     1.319432    23.179171
std       8.586941    13.725428    48.580373     6.743016    73.527325
min       0.000000     0.000000     0.000000     0.000000     1.000000
25%       0.000000     0.000000     1.000000     0.000000     1.000000
50%       1.000000     1.000000     2.000000     0.000000     3.000000
75%       2.000000     4.000000     7.000000     0.000000    13.000000
max     226.000000   284.000000  1050.000000   178.000000  1408.000000

In [15]:
# make work-rating multinomial distribution
# prior is Dirichlet distribution

alpha_rate = np.array([2.727866,5.148416,13.983458,1.319432]) * 0.05

WR_dist = pt_WR.drop('count', axis=1).apply(lambda x:x+alpha_rate, axis=1)

count = pt_WR['count']

for i in ['rate_w_0','rate_w_1','rate_w_2', 'rate_w_3']:
    WR_dist[i] = WR_dist[i] / (count+np.sum(alpha_rate))

WR_dist.head()

rate_w_0  rate_w_1  rate_w_2  rate_w_3
work_id                                        
0        0.298422  0.315328  0.377034  0.009215
1        0.063176  0.119234  0.787034  0.030557
2        0.091131  0.154969  0.679257  0.074644
3        0.022146  0.204161  0.762982  0.010711
4        0.140933  0.214884  0.573863  0.070320

### 2.2.5 Correspondence of work_id-img(scraping data), work_id-category_id

In [16]:
W_img_category = unsuper[['work_id','img', 'category_id']].drop_duplicates().sort_values(by=['work_id'])
W_img_category.index = W_img_category['work_id']
W_img_category.drop(['work_id'], axis=1, inplace=True)
W_img_category.head()

img  category_id
work_id                  
0          1            1
1          0            2
2          1            1
3          0            2
4          1            1

## 3.Merge FE features

In [17]:
X1 = train['user_id'].apply(lambda x: UW_ID.loc[x]    if x in UW_ID.index    else  UW_ID.describe().iloc[1])
X2 = train['user_id'].apply(lambda x: URC_dist.loc[x] if x in URC_dist.index else  URC_dist.describe().iloc[1])
X3 = train['work_id'].apply(lambda x: WR_dist.loc[x]  if x in WR_dist.index  else  WR_dist.describe().iloc[1])
X4 = train['work_id'].apply(lambda x: W_img_category.loc[x] if x in W_img_category.index else  W_img_category.describe().iloc[1])

X = pd.concat([X1, X2, X3, X4], axis=1)

y = train['rating']
y.name = 'rating'
train_data = pd.concat([X, y], axis=1)

In [18]:
X1 = test['user_id'].apply(lambda x: UW_ID.loc[x]    if x in UW_ID.index    else  UW_ID.describe().iloc[1])
X2 = test['user_id'].apply(lambda x: URC_dist.loc[x] if x in URC_dist.index else  URC_dist.describe().iloc[1])
X3 = test['work_id'].apply(lambda x: WR_dist.loc[x]  if x in WR_dist.index  else  WR_dist.describe().iloc[1])
X4 = test['work_id'].apply(lambda x: W_img_category.loc[x] if x in W_img_category.index else  W_img_category.describe().iloc[1])

test_data = pd.concat([X1, X2, X3, X4], axis=1)

## 4.save

In [19]:
train_data.to_csv("train_fe_finish.txt")

test_data.to_csv("test_fe_finish.txt")